<h3>Generation of SALCs in Octahedral and Tetrahedral Coordination Environments</h3>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

<h5>Local Imports</h5>

In [2]:
from SALC_functions import *

Octahedral Symmetry Group, $O_h$

The operations which compose the octahedral point group, $O_h$, of an octahedron with vertices at integer positions nearest to the origin as follows

$$(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)$$

Will be stored in a dictionary for later use.

In [3]:
### Vertices of the Octahedron ###

octahedral_vertices = np.array([[1,  0, -1,  0,  0,  0],
                                [0,  1,  0, -1,  0,  0],
                                [0,  0,  0,  0,  1, -1]], dtype=np.float64)



### The group symmetry elements as a list of the conjugacy classes of the group ###

#Identity
E = [np.eye(3)]

#180° rotation about the pricipal axes
C_2_4 = [rotation([ 1, 0, 0], np.pi),
         rotation([ 0, 1, 0], np.pi), 
         rotation([ 0, 0, 1], np.pi)]

#90° rotations about the principal axes
C_1_4 = [rotation([ 1, 0, 0], np.pi/2), 
         rotation([ 0, 1, 0], np.pi/2), 
         rotation([ 0, 0, 1], np.pi/2), 
         rotation([ 1, 0, 0],-np.pi/2), 
         rotation([ 0, 1, 0],-np.pi/2), 
         rotation([ 0, 0, 1],-np.pi/2)]

#180° rotation about the bisectors of the edges
C_2_prime = [rotation([ 1, 1, 0], np.pi), 
             rotation([-1, 1, 0], np.pi), 
             rotation([ 1, 0, 1], np.pi), 
             rotation([ 0, 1, 1], np.pi), 
             rotation([-1, 0, 1], np.pi), 
             rotation([ 0,-1, 1], np.pi)]

#120° rotation about the triangular face normal
C_1_3 = [rotation([ 1, 1, 1], 2*np.pi/3), 
         rotation([-1, 1, 1], 2*np.pi/3), 
         rotation([-1,-1, 1], 2*np.pi/3), 
         rotation([ 1,-1, 1], 2*np.pi/3), 
         rotation([ 1, 1, 1],-2*np.pi/3), 
         rotation([-1, 1, 1],-2*np.pi/3), 
         rotation([-1,-1, 1],-2*np.pi/3), 
         rotation([ 1,-1, 1],-2*np.pi/3)]

#Inversion
i = [-np.eye(3)]

#Inversion, then 180° rotation about the pricipal axes
iC_2_4 = [-np.eye(3) @ element for element in C_2_4]

#Inversion, then 90° rotations about the principal axes
iC_1_4 = [-np.eye(3) @ element for element in C_1_4]

#Inversion, 180° rotation about the bisectors of the edges
iC_2_prime = [-np.eye(3) @ element for element in C_2_prime]

#Inversion, 120° rotation about the triangular face normal
iC_1_3 = [-np.eye(3) @ element for element in C_1_3]

oct_classes = [E, C_2_4, C_1_4, C_2_prime, C_1_3, i, iC_2_4, iC_1_4, iC_2_prime, iC_1_3]
oct_class_size = [len(cls) for cls in oct_classes]
oct_order = sum(oct_class_size)

group_elements = []
for cls in oct_classes:
    for element in cls:
        group_elements.append(element)

#Check that none of the group elements are equal
for i in range(oct_order):
    for j in range(i+1, oct_order):
        if array_equal(group_elements[i], group_elements[j]):
            print("Elements {i}, {j} are equal".format(i=i, j=j))

#Check the closure of the group
for i in range(oct_order):
    for j in range(oct_order):
        product = group_elements[i] @ group_elements[j]
        included = False
        for k in range(oct_order):
            if array_equal(product, group_elements[k]):
                included = True
                break
        if not included:
            print("The product of elements {i} and {j} is not in the group".format(i=i, j=j))

#Check that the vertices are invariant under the group actions
for i in range(oct_order):
    transformed_vertices = group_elements[i] @ octahedral_vertices
    p = permutation_matrix(octahedral_vertices, transformed_vertices)
    if not valid_permutation_matrix(p):
        print("Element {i} does not leave the vertices invariant".format(i=i))



            

### Octahedral Character Table (Dresselhaus Table A.31)###

oct_char_table = {
    "A_plus_1"  : np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
    "A_plus_2"  : np.array([1, 1,-1,-1, 1, 1, 1,-1,-1, 1]),
    "E_plus"    : np.array([2, 2, 0, 0,-1, 2, 2, 0, 0,-1]),
    "T_minus_1" : np.array([3,-1, 1,-1, 0,-3, 1,-1, 1, 0]),
    "T_minus_2" : np.array([3,-1,-1, 1, 0,-3, 1, 1,-1, 0]),
    "A_minus_1" : np.array([1, 1, 1, 1, 1,-1,-1,-1,-1,-1]),
    "A_minus_2" : np.array([1, 1,-1,-1, 1,-1,-1, 1, 1,-1]),
    "E_minus"   : np.array([2, 2, 0, 0,-1,-2,-2, 0, 0, 1]),
    "T_plus_1"  : np.array([3,-1, 1,-1, 0, 3,-1, 1,-1, 0]),
    "T_plus_2"  : np.array([3,-1,-1, 1, 0, 3,-1,-1, 1, 0]),
}
#dimensionality of the IRs
oct_l = [1,1,2,3,3,1,1,2,3,3]


Tetrahedral Symmetry Group, $T_d$

The operations which compose the tetrahedral point group, $T_d$, of a tetrahedron with the following vertices

$$(1, 0, -\frac{\sqrt{2}}{2}), (-1, 0, -\frac{\sqrt{2}}{2}), (0, 1, \frac{\sqrt{2}}{2}), (0, -1, \frac{\sqrt{2}}{2})$$


In [4]:
### Vertices of the Tetrahedron ###

tetrahedral_vertices = np.array([[            1,           -1,            0,            0],
                                 [            0,            0,            1,           -1],
                                 [-np.sqrt(2)/2,-np.sqrt(2)/2, np.sqrt(2)/2, np.sqrt(2)/2]], dtype=np.float64)





### The group symmetry elements as a list of the conjugacy classes of the group ###

#Identity
E = [np.eye(3)]

#120° rotations about the vertices
C_1_3 = [rotation([ 1, 0, -np.sqrt(2)/2], 2*np.pi/3),
         rotation([-1, 0, -np.sqrt(2)/2], 2*np.pi/3),
         rotation([ 0, 1,  np.sqrt(2)/2], 2*np.pi/3),
         rotation([ 0,-1,  np.sqrt(2)/2], 2*np.pi/3),
         rotation([ 1, 0, -np.sqrt(2)/2], 4*np.pi/3),
         rotation([-1, 0, -np.sqrt(2)/2], 4*np.pi/3),
         rotation([ 0, 1,  np.sqrt(2)/2], 4*np.pi/3),
         rotation([ 0,-1,  np.sqrt(2)/2], 4*np.pi/3)]

#180° rotation about the edge bisectors
C_1_2 = [rotation([ 0, 0, 1], np.pi),
         rotation([ 1, 1, 0], np.pi),
         rotation([-1, 1, 0], np.pi)]

#mirror reflections
sigma_d = [reflection([ 0, 1, 0]),
           reflection([ np.sqrt(2),-np.sqrt(2), 2]),
           reflection([ np.sqrt(2), np.sqrt(2), 2]),
           reflection([ np.sqrt(2), np.sqrt(2),-2]),
           reflection([-np.sqrt(2), np.sqrt(2), 2]),
           reflection([ 1, 0, 0])]

#90° roto-reflection about the edge bisectors
S_1_4 = [rotation([ 0, 0, 1], np.pi/2)@reflection([ 0, 0, 1]),
         rotation([ 1, 1, 0], np.pi/2)@reflection([ 1, 1, 0]),
         rotation([-1, 1, 0], np.pi/2)@reflection([-1, 1, 0]),
         rotation([ 0, 0, 1], 3*np.pi/2)@reflection([ 0, 0, 1]),
         rotation([ 1, 1, 0], 3*np.pi/2)@reflection([ 1, 1, 0]),
         rotation([-1, 1, 0], 3*np.pi/2)@reflection([-1, 1, 0])]

tet_classes = [E, C_1_3, C_1_2, sigma_d, S_1_4]
tet_class_size = [len(cls) for cls in tet_classes]
tet_order = sum(tet_class_size)

group_elements = []
for cls in tet_classes:
    for element in cls:
        group_elements.append(element)

#Check that none of the group elements are equal
for i in range(tet_order):
    for j in range(i+1, tet_order):
        if array_equal(group_elements[i], group_elements[j]):
            print("Elements {i}, {j} are equal".format(i=i, j=j))

#Check the closure of the group
for i in range(tet_order):
    for j in range(tet_order):
        product = group_elements[i] @ group_elements[j]
        included = False
        for k in range(tet_order):
            if array_equal(product, group_elements[k]):
                included = True
                break
        if not included:
            print("The product of elements {i} and {j} is not in the group".format(i=i, j=j))
            print(np.around(product, 3))

#Check that the vertices are invariant under the group actions
for i in range(tet_order):
    transformed_vertices = group_elements[i] @ tetrahedral_vertices
    p = permutation_matrix(tetrahedral_vertices, transformed_vertices)
    if not valid_permutation_matrix(p):
        print("Element {i} does not leave the vertices invariant".format(i=i))




### Tetrahedral Character Table (Dresselhaus Table A.32)###

tet_char_table = {
    "A_1" : np.array([1, 1, 1, 1, 1]),
    "A_2" : np.array([1, 1, 1,-1,-1]),
    "E"   : np.array([2,-1, 2, 0, 0]),
    "T_1" : np.array([3, 0,-1,-1, 1]),
    "T_2" : np.array([3, 0,-1, 1,-1]),
}
#dimensionality of the IRs
tet_l  = [1,1,2,3,3]

<h3>S-orbital Representations</h3>

Since S-orbitals have spherical symmetry they can be modeled as points. The S-orbitals at each of the vertices can therefore be represented as vectors which point to the vertex on which the S-orbital is located.

In [5]:
def array_equal(a1: np.array, a2: np.array, eps=0.0001) -> np.array:
    """Check if two arrays are equal up to a accuracy factor eps"""
    diff = np.abs(a1 - a2)
    return np.all(diff < eps)


def permutation_matrix(vertices, transformed_vertices):
    """generate a permutation matrix associated with a transformation matrix acting on a set of vertices"""
    _, cols = vertices.shape
    perm_mat = np.zeros((cols, cols), dtype=np.int8)
    for i in range(cols):
        for j in range(cols):
            if array_equal(vertices[:,i], transformed_vertices[:,j]):
                perm_mat[i,j] = 1
                break
    return perm_mat


def point_representation(element : np.ndarray, vertices : np.ndarray) -> np.ndarray:
    """generate a list of representations of the group elements based on their action of the vertices, return as list of permuation matrices"""
    transformed_vertices = element @ vertices
    return permutation_matrix(vertices, transformed_vertices)
      

<h5>Octahedral S-orbital SALCs</h5>

In [6]:
#calculate the degeneracy of the SALCs in each irreducible group
for IR_key, IR_chars in oct_char_table.items():

    rep_chars = []
    for cls in oct_classes:
        rep = point_representation(cls[0], octahedral_vertices)
        rep_chars.append(np.trace(rep))

    rep_chars = np.array(rep_chars)
    row = np.multiply(oct_class_size, IR_chars)
    num = int((1/oct_order)*np.dot(row, rep_chars))
    print("{n} SALC(s) have {sym} symmetry".format(n=num, sym=IR_key))

1 SALC(s) have A_plus_1 symmetry
0 SALC(s) have A_plus_2 symmetry
1 SALC(s) have E_plus symmetry
1 SALC(s) have T_minus_1 symmetry
0 SALC(s) have T_minus_2 symmetry
0 SALC(s) have A_minus_1 symmetry
0 SALC(s) have A_minus_2 symmetry
0 SALC(s) have E_minus symmetry
0 SALC(s) have T_plus_1 symmetry
0 SALC(s) have T_plus_2 symmetry


Defining the Projection Operator for Points

In [7]:
def point_proj_operator(IR_key: str, class_list: list, char_table: dict, IR_dim: list, vertices) -> np.ndarray:
    """generate the projection operator"""
    _, basis_dim = vertices.shape
    proj_operator = np.zeros((basis_dim,basis_dim), dtype=np.float64)
    h = sum([len(cls) for cls in class_list])
    for i, cls in enumerate(class_list):
        for element in cls:
            proj_operator += char_table[IR_key][i] * point_representation(element, vertices)
    for i, key in enumerate(char_table.keys()):
        if key == IR_key:
            n = i
    proj_operator *= (IR_dim[n]/h)
    return proj_operator
    

Finding SALCs belonging to the $A_1^{+}$ irreducible group

In [13]:
proj_operator = point_proj_operator("A_plus_1", oct_classes, oct_char_table, oct_l, octahedral_vertices)
proj_basis = proj_operator @ np.eye(6)
print("The basis set under the action of the projection operator")
print(np.around(proj_basis,3), "\n")
orthonormal_basis = gram_schmidt(np.hsplit(proj_basis, 6))
print("The unique orthonormal basis consists of the following")
for v in orthonormal_basis:
    print(v)

The basis set under the action of the projection operator
[[0.167 0.167 0.167 0.167 0.167 0.167]
 [0.167 0.167 0.167 0.167 0.167 0.167]
 [0.167 0.167 0.167 0.167 0.167 0.167]
 [0.167 0.167 0.167 0.167 0.167 0.167]
 [0.167 0.167 0.167 0.167 0.167 0.167]
 [0.167 0.167 0.167 0.167 0.167 0.167]] 

The unique orthonormal basis consists of the following
[0.40824829 0.40824829 0.40824829 0.40824829 0.40824829 0.40824829]


Finding SALCs belonging to the $E^{+}$ irreducible group

In [19]:
proj_operator = point_proj_operator("E_plus", oct_classes, oct_char_table, oct_l, octahedral_vertices)
proj_basis = proj_operator @ np.eye(6)
print("The basis set under the action of the projection operator")
print(np.around(proj_basis,3), "\n")
orthonormal_basis = gram_schmidt(np.hsplit(proj_basis, 6))
print("The unique orthonormal basis consists of the following")
for v in orthonormal_basis:
    print(v)

The basis set under the action of the projection operator
[[ 0.333 -0.167  0.333 -0.167 -0.167 -0.167]
 [-0.167  0.333 -0.167  0.333 -0.167 -0.167]
 [ 0.333 -0.167  0.333 -0.167 -0.167 -0.167]
 [-0.167  0.333 -0.167  0.333 -0.167 -0.167]
 [-0.167 -0.167 -0.167 -0.167  0.333  0.333]
 [-0.167 -0.167 -0.167 -0.167  0.333  0.333]] 

The unique orthonormal basis consists of the following
[ 0.57735027 -0.28867513  0.57735027 -0.28867513 -0.28867513 -0.28867513]
[ 0.   0.5  0.   0.5 -0.5 -0.5]


Finding SALCs belonging to the $T_1^{-}$ irreducible group

In [20]:
proj_operator = point_proj_operator("T_minus_1", oct_classes, oct_char_table, oct_l, octahedral_vertices)
proj_basis = proj_operator @ np.eye(6)
print("The basis set under the action of the projection operator")
print(np.around(proj_basis,3), "\n")
orthonormal_basis = gram_schmidt(np.hsplit(proj_basis, 6))
print("The unique orthonormal basis consists of the following")
for v in orthonormal_basis:
    print(v)

The basis set under the action of the projection operator
[[ 0.5  0.  -0.5  0.   0.   0. ]
 [ 0.   0.5  0.  -0.5  0.   0. ]
 [-0.5  0.   0.5  0.   0.   0. ]
 [ 0.  -0.5  0.   0.5  0.   0. ]
 [ 0.   0.   0.   0.   0.5 -0.5]
 [ 0.   0.   0.   0.  -0.5  0.5]] 

The unique orthonormal basis consists of the following
[ 0.70710678  0.         -0.70710678  0.          0.          0.        ]
[ 0.          0.70710678  0.         -0.70710678  0.          0.        ]
[ 0.          0.          0.          0.          0.70710678 -0.70710678]


<h5>Tetrahedral S-orbital SALCs</h5>

In [140]:
#calculate the degeneracy of the SALCs in each irreducible group
for IR_key, IR_chars in tet_char_table.items():

    rep_chars = []
    for cls in tet_classes:
        rep = point_representation(cls[0], tetrahedral_vertices)
        rep_chars.append(np.trace(rep))

    rep_chars = np.array(rep_chars)
    row = np.multiply(tet_class_size, IR_chars)
    num = int((1/tet_order)*np.dot(row, rep_chars))
    print("{n} SALC(s) have {sym} symmetry".format(n=num, sym=IR_key))

1 SALC(s) have A_1 symmetry
0 SALC(s) have A_2 symmetry
0 SALC(s) have E symmetry
0 SALC(s) have T_1 symmetry
1 SALC(s) have T_2 symmetry


Finding SALCs belonging to the $A_1$ irreducible group

In [21]:
proj_operator = point_proj_operator("A_1", tet_classes, tet_char_table, tet_l, tetrahedral_vertices)
proj_basis = proj_operator @ np.eye(4)
print("The basis set under the action of the projection operator")
print(np.around(proj_basis,3), "\n")
orthonormal_basis = gram_schmidt(np.hsplit(proj_basis, 4))
print("The unique orthonormal basis consists of the following")
for v in orthonormal_basis:
    print(v)

The basis set under the action of the projection operator
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]] 

The unique orthonormal basis consists of the following
[0.5 0.5 0.5 0.5]


Finding SALCs belonging to the $T_2$ irreducible group

In [22]:
proj_operator = point_proj_operator("T_2", tet_classes, tet_char_table, tet_l, tetrahedral_vertices)
proj_basis = proj_operator @ np.eye(4)
print("The basis set under the action of the projection operator")
print(np.around(proj_basis,3), "\n")
orthonormal_basis = gram_schmidt(np.hsplit(proj_basis, 4))
print("The unique orthonormal basis consists of the following")
for v in orthonormal_basis:
    print(np.around(v,3))

The basis set under the action of the projection operator
[[ 0.75 -0.25 -0.25 -0.25]
 [-0.25  0.75 -0.25 -0.25]
 [-0.25 -0.25  0.75 -0.25]
 [-0.25 -0.25 -0.25  0.75]] 

The unique orthonormal basis consists of the following
[ 0.866 -0.289 -0.289 -0.289]
[-0.     0.816 -0.408 -0.408]
[-0.     0.     0.707 -0.707]


<h3>P-orbital Representations</h3>

P-orbitals transform as vectors located at each of the vertices.

In [26]:
def vector_representation(element : np.ndarray, vertices : np.ndarray) -> np.ndarray:
    """generate representations of the group elements based on their action on the vectors located at each vertex"""
    _, n = vertices.shape
    v = [np.array([1,0,0], dtype=np.float64),
         np.array([0,1,0], dtype=np.float64),
         np.array([0,0,1], dtype=np.float64)]

    rep = np.zeros((3*n, 3*n), dtype=np.float64)
    transformed_vertices = element @ vertices
    p = permutation_matrix(vertices, transformed_vertices)
    for i in range(n):
        for j in range(n):
            if p[i,j]:
                for k in range(3):
                    for l in range(3):
                        rep[3*i+k, 3*j+l] = np.dot(v[k], np.dot(element, v[l]))
    return rep


<h5>Octahedral P-orbital SALCs</h5>

In [27]:
#calculate the degeneracy of the SALCs in each irreducible group
for IR_key, IR_chars in oct_char_table.items():

    rep_chars = []
    for cls in oct_classes:
        rep = vector_representation(cls[0], octahedral_vertices)
        rep_chars.append(np.trace(rep))

    rep_chars = np.array(rep_chars)
    row = np.multiply(oct_class_size, IR_chars)
    num = int(np.around((1/oct_order)*np.dot(row, rep_chars), 3))
    print("{n} SALC(s) have {sym} symmetry".format(n=num, sym=IR_key))

1 SALC(s) have A_plus_1 symmetry
0 SALC(s) have A_plus_2 symmetry
1 SALC(s) have E_plus symmetry
2 SALC(s) have T_minus_1 symmetry
1 SALC(s) have T_minus_2 symmetry
0 SALC(s) have A_minus_1 symmetry
0 SALC(s) have A_minus_2 symmetry
0 SALC(s) have E_minus symmetry
1 SALC(s) have T_plus_1 symmetry
1 SALC(s) have T_plus_2 symmetry


In [28]:
def vector_proj_operator(IR_key: str, class_list: list, char_table: dict, IR_dim: list, vertices) -> np.ndarray:
    """generate the projection operator"""
    _, basis_dim = vertices.shape
    proj_operator = np.zeros((3*basis_dim,3*basis_dim), dtype=np.float64)
    h = sum([len(cls) for cls in class_list])
    for i, cls in enumerate(class_list):
        for element in cls:
            proj_operator += char_table[IR_key][i] * vector_representation(element, vertices)
    for i, key in enumerate(char_table.keys()):
        if key == IR_key:
            n = i
    proj_operator *= (IR_dim[n]/h)
    return proj_operator

In [57]:
proj_operator = vector_proj_operator("A_plus_1", oct_classes, oct_char_table, oct_l, octahedral_vertices)

for i in range(6):
    for j in range(6):
        print(i,j)
        print(np.around(proj_operator[3*i:3*i+3,3*j:3*j+3],1), "\n")


0 0
[[ 0.2  0.  -0. ]
 [-0.   0.  -0. ]
 [ 0.   0.   0. ]] 

0 1
[[ 0.   0.2  0. ]
 [-0.   0.   0. ]
 [ 0.   0.   0. ]] 

0 2
[[-0.2 -0.   0. ]
 [ 0.  -0.   0. ]
 [-0.  -0.  -0. ]] 

0 3
[[ 0.  -0.2  0. ]
 [ 0.   0.   0. ]
 [ 0.  -0.   0. ]] 

0 4
[[ 0.   0.   0.2]
 [ 0.   0.  -0. ]
 [-0.   0.   0. ]] 

0 5
[[ 0.   0.  -0.2]
 [ 0.   0.   0. ]
 [ 0.   0.   0. ]] 

1 0
[[ 0.  -0.   0. ]
 [ 0.2  0.  -0. ]
 [ 0.   0.   0. ]] 

1 1
[[ 0.   0.   0. ]
 [-0.   0.2  0. ]
 [-0.  -0.   0. ]] 

1 2
[[ 0.   0.   0. ]
 [-0.2  0.   0. ]
 [ 0.   0.   0. ]] 

1 3
[[-0.  -0.  -0. ]
 [ 0.  -0.2 -0. ]
 [ 0.   0.  -0. ]] 

1 4
[[ 0.   0.   0. ]
 [ 0.   0.   0.2]
 [ 0.  -0.   0. ]] 

1 5
[[ 0.   0.  -0. ]
 [ 0.   0.  -0.2]
 [ 0.   0.   0. ]] 

2 0
[[-0.2 -0.   0. ]
 [ 0.  -0.   0. ]
 [-0.  -0.  -0. ]] 

2 1
[[ 0.  -0.2  0. ]
 [ 0.   0.   0. ]
 [ 0.  -0.   0. ]] 

2 2
[[ 0.2  0.  -0. ]
 [-0.   0.  -0. ]
 [ 0.   0.   0. ]] 

2 3
[[ 0.   0.2  0. ]
 [-0.   0.   0. ]
 [ 0.   0.   0. ]] 

2 4
[[ 0.   0.  -0.2]
 [

<h5>Tetrahedral P-orbital SALCs</h5>

In [167]:
#calculate the degeneracy of the SALCs in each irreducible group
for IR_key, IR_chars in tet_char_table.items():

    rep_chars = []
    for cls in tet_classes:
        rep = vector_representation(cls[0], tetrahedral_vertices)
        rep_chars.append(np.trace(rep))

    rep_chars = np.array(rep_chars)
    row = np.multiply(tet_class_size, IR_chars)
    num = int(np.around((1/tet_order)*np.dot(row, rep_chars), 3))
    print("{n} SALC(s) with {sym} symmetry".format(n=num, sym=IR_key))

1 SALC(s) with A_1 symmetry
0 SALC(s) with A_2 symmetry
1 SALC(s) with E symmetry
1 SALC(s) with T_1 symmetry
2 SALC(s) with T_2 symmetry
